In [ ]:
# export
# default_exp unet
import torch
from torch import nn
from fastcore.dispatch import patch

In [ ]:
# export
import sys
sys.path.append('..')
from attention_unet.blocks import res_blocks, UnetBlock, ConvLayer, SqueezeExpand

In [ ]:
# export
class UNet(nn.Module): 
    
    channels = 32, 64, 128, 256, 512
    stride = 2, 2, 2, 2, 2
    blocks = 1, 2, 2, 2, 2
    n_layers = 3

    def __init__(self, in_c, n_classes, act=nn.ReLU, norm=nn.BatchNorm3d): 
        super(UNet, self).__init__()
        original_in_c = in_c
        for i in range(self.n_layers):
            setattr(self, f'down_layer_{i}', res_blocks(in_c, self.channels[i], 
                                                        self.stride[i], self.blocks[i], 
                                                        act=act, norm=norm))
            in_c = self.channels[i]
            
        self.middle_conv = SqueezeExpand(in_c, 0.2)
            
        for i in reversed(range(self.n_layers)): 
            s_c = self.channels[i-1] if i > 0 else original_in_c
            setattr(self, f'up_layer_{i}', UnetBlock(self.channels[i], s_c, act=act, norm=norm))
       
        self.final_conv = ConvLayer((s_c + self.channels[i]//2) // 2, n_classes, ks = 1)
        
        
    def forward(self, x): 
        sz = x.shape[-3:]
        # downsampling
        s = [x] # collect outouts for skip connections
        for i in range(self.n_layers): 
            x = getattr(self, f'down_layer_{i}')(x)
            s.append(x) 
        
        # middle conv missing
        x = self.middle_conv(x)

        # upsampling
        for i in reversed(range(self.n_layers)):
            x = getattr(self, f'up_layer_{i}')(x, s[i]) 
            # len(s) == (self.n_layers + 1), so s[i] will yield the right skip connection

        # final layer
        x = self.final_conv(x)
        
        # final resize
        if x.shape[-3:] != sz: 
            x = F.interpolate(x, sz, mode='nearest')
        return x
        

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted blocks.ipynb.
Converted unet.ipynb.
Converted utils.ipynb.
